In [1]:
from astropy.table import Table, join, hstack, unique
from astropy.io import ascii
from astropy.coordinates import SkyCoord
import astropy.units as u

import numpy as np

from glob import glob

import matplotlib as mpl
import matplotlib.pyplot as plt

import os
import requests

## Load SGA Data

In [2]:
#SGA = Table.read('/global/cfs/cdirs/cosmo/data/sga/2020/SGA-2020.fits', 'ELLIPSE')
SGA = Table.read('/Users/kellydouglass/Documents/Research/data/SGA/SGA-2020.fits', 'ELLIPSE')
SGA[:5]

SGA_ID,SGA_GALAXY,GALAXY,PGC,RA_LEDA,DEC_LEDA,MORPHTYPE,PA_LEDA,D25_LEDA,BA_LEDA,Z_LEDA,SB_D25_LEDA,MAG_LEDA,BYHAND,REF,GROUP_ID,GROUP_NAME,GROUP_MULT,GROUP_PRIMARY,GROUP_RA,GROUP_DEC,GROUP_DIAMETER,BRICKNAME,RA,DEC,D26,D26_REF,PA,BA,RA_MOMENT,DEC_MOMENT,SMA_MOMENT,G_SMA50,R_SMA50,Z_SMA50,SMA_SB22,SMA_SB22.5,SMA_SB23,SMA_SB23.5,SMA_SB24,SMA_SB24.5,SMA_SB25,SMA_SB25.5,SMA_SB26,G_MAG_SB22,R_MAG_SB22,Z_MAG_SB22,G_MAG_SB22.5,R_MAG_SB22.5,Z_MAG_SB22.5,G_MAG_SB23,R_MAG_SB23,Z_MAG_SB23,G_MAG_SB23.5,R_MAG_SB23.5,Z_MAG_SB23.5,G_MAG_SB24,R_MAG_SB24,Z_MAG_SB24,G_MAG_SB24.5,R_MAG_SB24.5,Z_MAG_SB24.5,G_MAG_SB25,R_MAG_SB25,Z_MAG_SB25,G_MAG_SB25.5,R_MAG_SB25.5,Z_MAG_SB25.5,G_MAG_SB26,R_MAG_SB26,Z_MAG_SB26,SMA_SB22_ERR,SMA_SB22.5_ERR,SMA_SB23_ERR,SMA_SB23.5_ERR,SMA_SB24_ERR,SMA_SB24.5_ERR,SMA_SB25_ERR,SMA_SB25.5_ERR,SMA_SB26_ERR,G_MAG_SB22_ERR,R_MAG_SB22_ERR,Z_MAG_SB22_ERR,G_MAG_SB22.5_ERR,R_MAG_SB22.5_ERR,Z_MAG_SB22.5_ERR,G_MAG_SB23_ERR,R_MAG_SB23_ERR,Z_MAG_SB23_ERR,G_MAG_SB23.5_ERR,R_MAG_SB23.5_ERR,Z_MAG_SB23.5_ERR,G_MAG_SB24_ERR,R_MAG_SB24_ERR,Z_MAG_SB24_ERR,G_MAG_SB24.5_ERR,R_MAG_SB24.5_ERR,Z_MAG_SB24.5_ERR,G_MAG_SB25_ERR,R_MAG_SB25_ERR,Z_MAG_SB25_ERR,G_MAG_SB25.5_ERR,R_MAG_SB25.5_ERR,Z_MAG_SB25.5_ERR,G_MAG_SB26_ERR,R_MAG_SB26_ERR,Z_MAG_SB26_ERR,G_COG_PARAMS_MTOT,G_COG_PARAMS_M0,G_COG_PARAMS_ALPHA1,G_COG_PARAMS_ALPHA2,G_COG_PARAMS_CHI2,R_COG_PARAMS_MTOT,R_COG_PARAMS_M0,R_COG_PARAMS_ALPHA1,R_COG_PARAMS_ALPHA2,R_COG_PARAMS_CHI2,Z_COG_PARAMS_MTOT,Z_COG_PARAMS_M0,Z_COG_PARAMS_ALPHA1,Z_COG_PARAMS_ALPHA2,Z_COG_PARAMS_CHI2,ELLIPSEBIT
int64,bytes16,bytes29,int64,float64,float64,bytes21,float32,float32,float32,float32,float32,float32,bool,bytes13,int64,bytes35,int16,bool,float64,float64,float32,bytes8,float64,float64,float32,bytes4,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32
2,SGA-2020 2,PGC1283207,1283207,228.3770865,5.4232017,S?,152.2,0.36307806,0.724436,0.03463229,23.40448,16.976,False,LEDA-20181114,0,PGC1283207,1,True,228.3770865,5.4232017,0.36307806,2283p055,228.3770803831908,5.423191398593787,0.49470574,SB26,158.20142,0.545691,228.37700918822188,5.4232652570544015,10.897086,3.3509698,3.1147978,3.240862,5.902337,6.9126143,7.941369,8.997992,10.073601,11.199986,12.391357,13.561038,14.841172,16.966799,16.108246,15.486356,16.879545,16.024958,15.400715,16.818878,15.967034,15.341793,16.776297,15.925804,15.300776,16.746685,15.897334,15.272053,16.725166,15.876816,15.2521105,16.708357,15.862035,15.237181,16.696539,15.851936,15.226998,16.689613,15.844313,15.21976,0.013392451,0.02354,0.021872982,0.01736985,0.024445537,0.039866067,0.05026544,0.08455789,0.122911856,0.005682776,0.0054258136,0.0049038026,0.005588406,0.005323561,0.0047632363,0.00543534,0.005177031,0.0046343105,0.0053025587,0.005040888,0.0045181247,0.005206092,0.0049438984,0.0044374703,0.0051483097,0.0048758644,0.0043834248,0.0051032505,0.0048264163,0.004344248,0.0050705094,0.004792021,0.004319857,0.005054293,0.004765629,0.0043044444,16.65942,0.34037337,0.2978292,3.0239506,0.07928849,15.820566,0.2640441,0.34559453,3.3033552,0.003811298,15.195567,0.29826432,0.3001073,3.2333765,0.011723555,0
3,SGA-2020 3,PGC1310416,1310416,202.54443750000002,6.9345944,Sc,159.26,0.4017908,0.7816278,0.073888786,23.498482,16.85,False,LEDA-20181114,1,PGC1310416,1,True,202.54443750000002,6.9345944,0.4017908,

# Match stellar distances

In [3]:
stellar_dist = glob('cache/stellar_dist/*.txt')

In [4]:
stellar_dist[0].split('/')[-1].strip('.txt').split('_')[-1]
#stellar_dist[0].split('/')[-1].strip('.txt').strip('EDDtable_')

'HSTIRSBF'

In [5]:
PGC_ids = []
DM = []
e_DM_hi = []
e_DM_lo = []
dist = []
e_dist = []
catalog = []

for sfile in stellar_dist:
    
    print(sfile)
    '''
    if sfile == 'cache/stellar_dist/EDDtable_ANGST.txt':
        continue
    '''
    # Catalog name
    #name = sfile.split('/')[-1].strip('.txt').strip('EDDtable_')
    name = sfile.split('/')[-1].strip('.txt').split('_')[-1]
    
    # Read in data file
    tab = Table.read(sfile, format='csv', header_start=1, data_start=5)
    
    # Rename the pgc column
    tab.rename_column('pgc', 'PGC')
    
    # Join this distance catalog with the SGA catalog
    jtab = join(SGA, tab, keys='PGC', join_type='inner')
    
    print(tab.colnames)
    
    # Append this catalog's name to the list of all names
    catalog.append(np.full(np.shape(jtab['PGC'].flatten()), name))
    if name == 'Araucaria':
        for i in range(4):
            catalog.append(np.full(np.shape(jtab['PGC'].flatten()), name))
    
    # Append this catalog's list of PGC numbers to the list of all PGC numbers with distances
    PGC_ids.append(jtab['PGC'].flatten())
    if name == 'Araucaria':
        for i in range(4):
            PGC_ids.append(jtab['PGC'].flatten())
    
    # Append distance moduli to list of all distance moduli
    if name == 'Araucaria':
        DM.append(jtab['IRCeph'].flatten())
        DM.append(jtab['IRRRLy'].flatten())
        DM.append(jtab['TRGB'].flatten())
        DM.append(jtab['FGLR'].flatten())
        DM.append(jtab['EB'].flatten())
    else:
        try:
            DM.append(jtab['DM'].flatten())
        except KeyError:
            try:
                DM.append(jtab['dm'].flatten())
            except KeyError:
                try:
                    DM.append(jtab['Mod01'].flatten())
                except KeyError:
                    try:
                        DM.append(jtab['DM_tip'].flatten())
                    except KeyError:
                        try:
                            DM.append(jtab['modulus'].flatten())
                        except KeyError:
                            print('DM not found')
    
    # Append uncertainty in the distance moduli to the list of all uncertainties
    if name == 'Araucaria':
        e_DM_hi.append(np.sqrt(jtab['erC'].flatten()**2 + jtab['esC'].flatten()**2))
        e_DM_hi.append(np.sqrt(jtab['erR'].flatten()**2 + jtab['esR'].flatten()**2))
        e_DM_hi.append(np.sqrt(jtab['erT'].flatten()**2 + jtab['esT'].flatten()**2))
        e_DM_hi.append(jtab['eFGLR'].flatten())
        e_DM_hi.append(jtab['eEB'].flatten())
    else:
        try:
            e_DM_hi.append(jtab['e_dm'].flatten())
        except KeyError:
            try:
                e_DM_hi.append(jtab['edm'].flatten())
            except KeyError:
                try:
                    e_DM_hi.append(jtab['e_Mod'].flatten())
                except KeyError:
                    try:
                        e_DM_hi.append(jtab['eDM_hi'].flatten())
                    except KeyError:
                        try:
                            e_DM_hi.append(jtab['eDM+'].flatten())
                        except KeyError:
                            e_DM_hi.append(np.zeros_like(jtab['modulus'].flatten()))
                            print('E_DM_hi not found')
    
    # Append uncertainty in the distance moduli to the list of all uncertainties
    if name == 'Araucaria':
        e_DM_lo.append(np.sqrt(jtab['erC'].flatten()**2 + jtab['esC'].flatten()**2))
        e_DM_lo.append(np.sqrt(jtab['erR'].flatten()**2 + jtab['esR'].flatten()**2))
        e_DM_lo.append(np.sqrt(jtab['erT'].flatten()**2 + jtab['esT'].flatten()**2))
        e_DM_lo.append(jtab['eFGLR'].flatten())
        e_DM_lo.append(jtab['eEB'].flatten())
    else:
        try:
            e_DM_lo.append(jtab['e_dm'].flatten())
        except KeyError:
            try:
                e_DM_lo.append(jtab['edm'].flatten())
            except KeyError:
                try:
                    e_DM_lo.append(jtab['e_Mod'].flatten())
                except KeyError:
                    try:
                        e_DM_lo.append(jtab['eDM_lo'].flatten())
                    except KeyError:
                        try:
                            e_DM_lo.append(jtab['eDM-'].flatten())
                        except KeyError:
                            e_DM_lo.append(np.zeros_like(jtab['modulus'].flatten()))
                            print('E_DM_lo not found')
                        
#     try:
#         dist.append(jtab['dist'].flatten())
#     except KeyError:
#         try:
#             DM.append(jtab['dm'].flatten())
#         except KeyError:
#             try:
#                 DM.append(jtab['Mod01'].flatten())
#             except KeyError:
#                 try:
#                     DM.append(jtab['DM_tip'].flatten())
#                 except KeyError:
#                     try:
#                         DM.append(jtab['modulus'].flatten())
#                     except KeyError:
#                         print('DM not found')
# #     try:
#         e_DM.append(jtab['e_dm'].flatten())
#     except KeyError:
#         try:
#             e_DM.append(jtab['edm'].flatten())
#         except KeyError:
#             print('e_dm not found')
    # print()
#     dist.append(jtab['dist'].flatten())
#     e_dist.append(jtab['e_d'].flatten())
    
    # Check that the list lengths are all the same
    if not all([len(catalog), len(PGC_ids), len(DM), len(e_DM_hi), len(e_DM_lo)]):
        print('Not all lists are the same length!')
        break
    
    print('Iteration complete')
    print()
    # print(jtab)
    # break

cache/stellar_dist/EDDtable_HSTIRSBF.txt
['PGC', 'Name', '1PGC', 'Nest', 'DMzp', 'DM', 'edm', 'Vhel', 'Ty', 'RA_DEJ', 'glon', 'glat', 'sgl', 'sgb', 'Ag']
Iteration complete

cache/stellar_dist/EDDtable_ANGST.txt
['PGC', 'Cat', 'Targ', 'Filt', 'Nstar', 'A_V', 'Color', 'Mtrgb', 'mraw', 'm_trgb', 'etrgb', 'modulus', 'dist', 'edist', 'Ncmd', 'avdist', 'eavdist']
E_DM_hi not found
E_DM_lo not found
Iteration complete

cache/stellar_dist/EDDtable_CMDsTRGB.txt
['PGC', 'Name/CMD', 'Alt_Source', 'D814', 'HST-pgm', 'Cam', 'D110', 'D160', 'HST-IR', 'Cam-IR', 'T814', 'T8_lo', 'T8_hi', '606-814', 'eb_lo', 'eb_hi', '555-814', 'ea_lo', 'ea_hi', 'TRGB', 'T_lo', 'T_hi', 'V-I', 'e_lo', 'e_hi', 'E1_B-V', 'E2_B-V', 'A_814', 'M_TRGB', 'DM_tip', 'eDM_lo', 'eDM_hi', 'T110', 'T1_lo', 'T1_hi', 'C110', 'e1_lo', 'e1_hi', 'T160', 'T6_lo', 'T6_hi', 'C160', 'e6_lo', 'e6_hi', 'A_110', 'A_160', 'M_T110', 'M_T160', 'DM_110', 'eDM1_lo', 'eDM1_hi', 'DM_160', 'eDM6_lo', 'eDM6_hi']
Iteration complete

cache/stellar_dist/E

In [6]:
def flatten_list(arr):
    flat_arr = []
    for i in range(len(arr)):
        for j in range(len(arr[i])):
            flat_arr.append(arr[i][j])
    return flat_arr

In [7]:
PGC_ids_flat = flatten_list(PGC_ids)
DM_flat = flatten_list(DM)
e_DM_lo_flat = flatten_list(e_DM_lo)
e_DM_hi_flat = flatten_list(e_DM_hi)
catalog_flat = flatten_list(catalog)

# for i in range(len(PGC_ids)):
#     for j in range(len(PGC_ids[i])):
#         PGC_ids_flat.append(PGC_ids[i][j])

In [8]:
len(PGC_ids_flat), len(DM_flat), len(e_DM_lo_flat), len(e_DM_hi_flat), len(catalog_flat)

(869, 869, 869, 869, 869)

In [9]:
tab = Table([PGC_ids_flat, DM_flat, e_DM_lo_flat, e_DM_hi_flat, catalog_flat],
            names=['PGC', 'DM_Stellar', 'e_DM_lo_Stellar', 'e_DM_hi_Stellar', 'Stellar_Catalog'])

tab

PGC,DM_Stellar,e_DM_lo_Stellar,e_DM_hi_Stellar,Stellar_Catalog
int64,float64,float64,float64,str14
1037,34.126,0.094,0.094,HSTIRSBF
3455,34.166,0.081,0.081,HSTIRSBF
3982,34.101,0.084,0.084,HSTIRSBF
5098,33.953,0.081,0.081,HSTIRSBF
5222,32.212,0.09,0.09,HSTIRSBF
5283,34.31,0.077,0.077,HSTIRSBF
5323,34.489,0.11,0.11,HSTIRSBF
5324,34.391,0.096,0.096,HSTIRSBF
6415,33.973,0.084,0.084,HSTIRSBF


In [10]:
tab['DM_Stellar'].fill_value = -1
tab['e_DM_hi_Stellar'].fill_value = -1
tab['e_DM_lo_Stellar'].fill_value = -1
tab['Stellar_Catalog'].fill_value = -1
tab = tab.filled()
tab

PGC,DM_Stellar,e_DM_lo_Stellar,e_DM_hi_Stellar,Stellar_Catalog
int64,float64,float64,float64,str14
1037,34.126,0.094,0.094,HSTIRSBF
3455,34.166,0.081,0.081,HSTIRSBF
3982,34.101,0.084,0.084,HSTIRSBF
5098,33.953,0.081,0.081,HSTIRSBF
5222,32.212,0.09,0.09,HSTIRSBF
5283,34.31,0.077,0.077,HSTIRSBF
5323,34.489,0.11,0.11,HSTIRSBF
5324,34.391,0.096,0.096,HSTIRSBF
6415,33.973,0.084,0.084,HSTIRSBF


In [11]:
utab = unique(tab, keys=['PGC', 'DM_Stellar', 'e_DM_lo_Stellar', 'e_DM_hi_Stellar'])
utab

PGC,DM_Stellar,e_DM_lo_Stellar,e_DM_hi_Stellar,Stellar_Catalog
int64,float64,float64,float64,str14
218,30.6,0.14,0.14,TonrySBF
218,30.78,30.69,30.88,CMDsTRGB
621,27.53,27.49,27.57,CMDsTRGB
1014,-1.0,-1.0,-1.0,Araucaria
1014,26.43,0.08944271909999159,0.08944271909999159,Araucaria
1014,26.43,0.12,0.12,McConnachie
1014,26.604,0.0,0.0,ANGST
1014,26.62,26.59,26.65,CMDsTRGB
1037,34.126,0.094,0.094,HSTIRSBF


In [12]:
# Also remove all table rows that are full of -1s (contain no data)
empty_rows = (utab['DM_Stellar'] == -1) & (utab['e_DM_lo_Stellar'] == -1) & (utab['e_DM_hi_Stellar'] == -1)
utab_noempty = utab[~empty_rows]
utab_noempty

PGC,DM_Stellar,e_DM_lo_Stellar,e_DM_hi_Stellar,Stellar_Catalog
int64,float64,float64,float64,str14
218,30.6,0.14,0.14,TonrySBF
218,30.78,30.69,30.88,CMDsTRGB
621,27.53,27.49,27.57,CMDsTRGB
1014,26.43,0.08944271909999159,0.08944271909999159,Araucaria
1014,26.43,0.12,0.12,McConnachie
1014,26.604,0.0,0.0,ANGST
1014,26.62,26.59,26.65,CMDsTRGB
1037,34.126,0.094,0.094,HSTIRSBF
1038,26.42,0.04,0.04,McConnachie


In [13]:
len(SGA)

383620

In [14]:
#maintab = join(SGA, utab, keys='PGC', join_type='left')
maintab = join(SGA, utab_noempty, keys='PGC', join_type='left')

maintab[:5]

SGA_ID,SGA_GALAXY,GALAXY,PGC,RA_LEDA,DEC_LEDA,MORPHTYPE,PA_LEDA,D25_LEDA,BA_LEDA,Z_LEDA,SB_D25_LEDA,MAG_LEDA,BYHAND,REF,GROUP_ID,GROUP_NAME,GROUP_MULT,GROUP_PRIMARY,GROUP_RA,GROUP_DEC,GROUP_DIAMETER,BRICKNAME,RA,DEC,D26,D26_REF,PA,BA,RA_MOMENT,DEC_MOMENT,SMA_MOMENT,G_SMA50,R_SMA50,Z_SMA50,SMA_SB22,SMA_SB22.5,SMA_SB23,SMA_SB23.5,SMA_SB24,SMA_SB24.5,SMA_SB25,SMA_SB25.5,SMA_SB26,G_MAG_SB22,R_MAG_SB22,Z_MAG_SB22,G_MAG_SB22.5,R_MAG_SB22.5,Z_MAG_SB22.5,G_MAG_SB23,R_MAG_SB23,Z_MAG_SB23,G_MAG_SB23.5,R_MAG_SB23.5,Z_MAG_SB23.5,G_MAG_SB24,R_MAG_SB24,Z_MAG_SB24,G_MAG_SB24.5,R_MAG_SB24.5,Z_MAG_SB24.5,G_MAG_SB25,R_MAG_SB25,Z_MAG_SB25,G_MAG_SB25.5,R_MAG_SB25.5,Z_MAG_SB25.5,G_MAG_SB26,R_MAG_SB26,Z_MAG_SB26,SMA_SB22_ERR,SMA_SB22.5_ERR,SMA_SB23_ERR,SMA_SB23.5_ERR,SMA_SB24_ERR,SMA_SB24.5_ERR,SMA_SB25_ERR,SMA_SB25.5_ERR,SMA_SB26_ERR,G_MAG_SB22_ERR,R_MAG_SB22_ERR,Z_MAG_SB22_ERR,G_MAG_SB22.5_ERR,R_MAG_SB22.5_ERR,Z_MAG_SB22.5_ERR,G_MAG_SB23_ERR,R_MAG_SB23_ERR,Z_MAG_SB23_ERR,G_MAG_SB23.5_ERR,R_MAG_SB23.5_ERR,Z_MAG_SB23.5_ERR,G_MAG_SB24_ERR,R_MAG_SB24_ERR,Z_MAG_SB24_ERR,G_MAG_SB24.5_ERR,R_MAG_SB24.5_ERR,Z_MAG_SB24.5_ERR,G_MAG_SB25_ERR,R_MAG_SB25_ERR,Z_MAG_SB25_ERR,G_MAG_SB25.5_ERR,R_MAG_SB25.5_ERR,Z_MAG_SB25.5_ERR,G_MAG_SB26_ERR,R_MAG_SB26_ERR,Z_MAG_SB26_ERR,G_COG_PARAMS_MTOT,G_COG_PARAMS_M0,G_COG_PARAMS_ALPHA1,G_COG_PARAMS_ALPHA2,G_COG_PARAMS_CHI2,R_COG_PARAMS_MTOT,R_COG_PARAMS_M0,R_COG_PARAMS_ALPHA1,R_COG_PARAMS_ALPHA2,R_COG_PARAMS_CHI2,Z_COG_PARAMS_MTOT,Z_COG_PARAMS_M0,Z_COG_PARAMS_ALPHA1,Z_COG_PARAMS_ALPHA2,Z_COG_PARAMS_CHI2,ELLIPSEBIT,DM_Stellar,e_DM_lo_Stellar,e_DM_hi_Stellar,Stellar_Catalog
int64,bytes16,bytes29,int64,float64,float64,bytes21,float32,float32,float32,float32,float32,float32,bool,bytes13,int64,bytes35,int16,bool,float64,float64,float32,bytes8,float64,float64,float32,bytes4,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,float64,float64,float64,str14
5002480,SGA-2020 5002480,DR8-2837p402-5033,-1,283.619631436471,40.3162008554537,DEV,178.80934,0.54342,0.3166,-1.0,24.29036,16.9862,False,DR8,530781,DR8-2837p402-5033,1,True,283.619631436471,40.3162008554537,0.54342,2837p402,283.6196056669463,40.316337272867564,0.53864765,SB26,179.53694,0.3690244,283.6194583471782,40.31653379127378,11.641381,11.157478,8.614751,9.8993,1.6142721,3.685578,5.5844307,7.701021,9.878292,11.927793,13.400182,14.697571,16.15943,21.37251,20.373438,19.779512,19.920574,19.005274,18.41066,19.314016,18.435898,17.843414,18.922651,18.076838,17.494484,18.664383,17.833265,17.25879,18.496494,17.682926,17.106598,18.404135,17.605488,17.02742,18.332602,17.555883,16.971588,18.299273,17.52499,16.931496,0.04933311,0.065890566,0.09742977,0.10495926,0.1614855,0.27832195,0.15179653,0.13904023,0.20484819,0.041657884,0.06669066,0.06785251,0.038538612,0.049561314,0.05101378,0.03635901,0.041818976,0.042013034,0.03367074,0.03763669,0.03705858,0.03122176,0.03420294,0.033616345,0.029198052,0.031941395,0.031132452,0.028080799,0.030865192,0.029955743,0.02722575,0.030135926,0.029090546,0.026788738,0.02959205,0.028436601,17.799881,1.3952494,0.83855844,1.4560318,0.5604667,17.200954,1.0272083,0.8301853,1.7677885,0.4513309,16.501709,1.3551984,0.73172045,1.4457989,0.2076136,0,--,--,--,--
5001827,SGA-2020 5001827,DR8-3429p190-2654,-1,343.06599510416885,19.007842544240166,COMP,149.72781,0.36448

In [15]:
maintab[maintab['PGC'] == 771919]

SGA_ID,SGA_GALAXY,GALAXY,PGC,RA_LEDA,DEC_LEDA,MORPHTYPE,PA_LEDA,D25_LEDA,BA_LEDA,Z_LEDA,SB_D25_LEDA,MAG_LEDA,BYHAND,REF,GROUP_ID,GROUP_NAME,GROUP_MULT,GROUP_PRIMARY,GROUP_RA,GROUP_DEC,GROUP_DIAMETER,BRICKNAME,RA,DEC,D26,D26_REF,PA,BA,RA_MOMENT,DEC_MOMENT,SMA_MOMENT,G_SMA50,R_SMA50,Z_SMA50,SMA_SB22,SMA_SB22.5,SMA_SB23,SMA_SB23.5,SMA_SB24,SMA_SB24.5,SMA_SB25,SMA_SB25.5,SMA_SB26,G_MAG_SB22,R_MAG_SB22,Z_MAG_SB22,G_MAG_SB22.5,R_MAG_SB22.5,Z_MAG_SB22.5,G_MAG_SB23,R_MAG_SB23,Z_MAG_SB23,G_MAG_SB23.5,R_MAG_SB23.5,Z_MAG_SB23.5,G_MAG_SB24,R_MAG_SB24,Z_MAG_SB24,G_MAG_SB24.5,R_MAG_SB24.5,Z_MAG_SB24.5,G_MAG_SB25,R_MAG_SB25,Z_MAG_SB25,G_MAG_SB25.5,R_MAG_SB25.5,Z_MAG_SB25.5,G_MAG_SB26,R_MAG_SB26,Z_MAG_SB26,SMA_SB22_ERR,SMA_SB22.5_ERR,SMA_SB23_ERR,SMA_SB23.5_ERR,SMA_SB24_ERR,SMA_SB24.5_ERR,SMA_SB25_ERR,SMA_SB25.5_ERR,SMA_SB26_ERR,G_MAG_SB22_ERR,R_MAG_SB22_ERR,Z_MAG_SB22_ERR,G_MAG_SB22.5_ERR,R_MAG_SB22.5_ERR,Z_MAG_SB22.5_ERR,G_MAG_SB23_ERR,R_MAG_SB23_ERR,Z_MAG_SB23_ERR,G_MAG_SB23.5_ERR,R_MAG_SB23.5_ERR,Z_MAG_SB23.5_ERR,G_MAG_SB24_ERR,R_MAG_SB24_ERR,Z_MAG_SB24_ERR,G_MAG_SB24.5_ERR,R_MAG_SB24.5_ERR,Z_MAG_SB24.5_ERR,G_MAG_SB25_ERR,R_MAG_SB25_ERR,Z_MAG_SB25_ERR,G_MAG_SB25.5_ERR,R_MAG_SB25.5_ERR,Z_MAG_SB25.5_ERR,G_MAG_SB26_ERR,R_MAG_SB26_ERR,Z_MAG_SB26_ERR,G_COG_PARAMS_MTOT,G_COG_PARAMS_M0,G_COG_PARAMS_ALPHA1,G_COG_PARAMS_ALPHA2,G_COG_PARAMS_CHI2,R_COG_PARAMS_MTOT,R_COG_PARAMS_M0,R_COG_PARAMS_ALPHA1,R_COG_PARAMS_ALPHA2,R_COG_PARAMS_CHI2,Z_COG_PARAMS_MTOT,Z_COG_PARAMS_M0,Z_COG_PARAMS_ALPHA1,Z_COG_PARAMS_ALPHA2,Z_COG_PARAMS_CHI2,ELLIPSEBIT,DM_Stellar,e_DM_lo_Stellar,e_DM_hi_Stellar,Stellar_Catalog
int64,bytes16,bytes29,int64,float64,float64,bytes21,float32,float32,float32,float32,float32,float32,bool,bytes13,int64,bytes35,int16,bool,float64,float64,float32,bytes8,float64,float64,float32,bytes4,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,float64,float64,float64,str14
868032,SGA-2020 868032,NGC1201,771919,46.03332532673745,-26.06969750695285,E-S0,8.59,3.411929,0.57411647,0.005596872,22.99448,11.701,True,LEDA-20181114,317178,NGC1201,1,True,46.03332532673745,-26.06969750695285,3.411929,0459m260,46.03332172733179,-26.06966853637713,4.9620147,SB26,7.90505,0.56788504,46.033208590750064,-26.069688000976626,118.01962,27.42394,27.187017,27.923996,61.19563,72.84604,83.99373,94.485725,104.98598,115.697334,126.560875,137.64,148.86044,11.54185,10.780679,10.202037,11.4601555,10.698883,10.118637,11.407081,10.646178,10.065561,11.373177,10.6126375,10.031863,11.349672,10.589578,10.0087805,11.333385,10.573862,9.993349,11.321908,10.562697,9.982237,11.313794,10.5551195,9.974818,11.308098,10.5498085,9.96952,0.02019594,0.030068478,0.020640232,0.026782066,0.027618866,0.043218754,0.053795904,0.06738641,0.08724375,0.0021383492,0.0025708186,0.0028877133,0.0019859343,0.0023212722,0.0026519478,0.0018893456,0.002271816,0.002547142,0.0018313518,0.0022027814,0.0024693797,0.0017941092,0.002099232,0.002397022,0.001765516,0.0021255494,0.0023833702,0.0017488543,0.0020479271,0.0023391622,0.0017339756,0.0020891912,0.0023430607,0.0017249141,0.002079005,0.0023316648,11.290278,0.18008748,2974.8523,3.800875,0.0053895037,10.53363,0.17351764,3786.2764,3.9145265,0.014138333,9.953629,0.18070917,3505.5095,3.9080532,0.021632774,0,31.347,0.074,0.074,HSTIRSBF
868032,SGA-2020 868032,NGC1201,771

In [16]:
maintab['DM_Stellar'].fill_value = -1
maintab['e_DM_hi_Stellar'].fill_value = -1
maintab['e_DM_lo_Stellar'].fill_value = -1
maintab['Stellar_Catalog'].fill_value = -1

maintab = maintab.filled()

maintab

SGA_ID,SGA_GALAXY,GALAXY,PGC,RA_LEDA,DEC_LEDA,MORPHTYPE,PA_LEDA,D25_LEDA,BA_LEDA,Z_LEDA,SB_D25_LEDA,MAG_LEDA,BYHAND,REF,GROUP_ID,GROUP_NAME,GROUP_MULT,GROUP_PRIMARY,GROUP_RA,GROUP_DEC,GROUP_DIAMETER,BRICKNAME,RA,DEC,D26,D26_REF,PA,BA,RA_MOMENT,DEC_MOMENT,SMA_MOMENT,G_SMA50,R_SMA50,Z_SMA50,SMA_SB22,SMA_SB22.5,SMA_SB23,SMA_SB23.5,SMA_SB24,SMA_SB24.5,SMA_SB25,SMA_SB25.5,SMA_SB26,G_MAG_SB22,R_MAG_SB22,Z_MAG_SB22,G_MAG_SB22.5,R_MAG_SB22.5,Z_MAG_SB22.5,G_MAG_SB23,R_MAG_SB23,Z_MAG_SB23,G_MAG_SB23.5,R_MAG_SB23.5,Z_MAG_SB23.5,G_MAG_SB24,R_MAG_SB24,Z_MAG_SB24,G_MAG_SB24.5,R_MAG_SB24.5,Z_MAG_SB24.5,G_MAG_SB25,R_MAG_SB25,Z_MAG_SB25,G_MAG_SB25.5,R_MAG_SB25.5,Z_MAG_SB25.5,G_MAG_SB26,R_MAG_SB26,Z_MAG_SB26,SMA_SB22_ERR,SMA_SB22.5_ERR,SMA_SB23_ERR,SMA_SB23.5_ERR,SMA_SB24_ERR,SMA_SB24.5_ERR,SMA_SB25_ERR,SMA_SB25.5_ERR,SMA_SB26_ERR,G_MAG_SB22_ERR,R_MAG_SB22_ERR,Z_MAG_SB22_ERR,G_MAG_SB22.5_ERR,R_MAG_SB22.5_ERR,Z_MAG_SB22.5_ERR,G_MAG_SB23_ERR,R_MAG_SB23_ERR,Z_MAG_SB23_ERR,G_MAG_SB23.5_ERR,R_MAG_SB23.5_ERR,Z_MAG_SB23.5_ERR,G_MAG_SB24_ERR,R_MAG_SB24_ERR,Z_MAG_SB24_ERR,G_MAG_SB24.5_ERR,R_MAG_SB24.5_ERR,Z_MAG_SB24.5_ERR,G_MAG_SB25_ERR,R_MAG_SB25_ERR,Z_MAG_SB25_ERR,G_MAG_SB25.5_ERR,R_MAG_SB25.5_ERR,Z_MAG_SB25.5_ERR,G_MAG_SB26_ERR,R_MAG_SB26_ERR,Z_MAG_SB26_ERR,G_COG_PARAMS_MTOT,G_COG_PARAMS_M0,G_COG_PARAMS_ALPHA1,G_COG_PARAMS_ALPHA2,G_COG_PARAMS_CHI2,R_COG_PARAMS_MTOT,R_COG_PARAMS_M0,R_COG_PARAMS_ALPHA1,R_COG_PARAMS_ALPHA2,R_COG_PARAMS_CHI2,Z_COG_PARAMS_MTOT,Z_COG_PARAMS_M0,Z_COG_PARAMS_ALPHA1,Z_COG_PARAMS_ALPHA2,Z_COG_PARAMS_CHI2,ELLIPSEBIT,DM_Stellar,e_DM_lo_Stellar,e_DM_hi_Stellar,Stellar_Catalog
int64,bytes16,bytes29,int64,float64,float64,bytes21,float32,float32,float32,float32,float32,float32,bool,bytes13,int64,bytes35,int16,bool,float64,float64,float32,bytes8,float64,float64,float32,bytes4,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,float64,float64,float64,str14
5002480,SGA-2020 5002480,DR8-2837p402-5033,-1,283.619631436471,40.3162008554537,DEV,178.80934,0.54342,0.3166,-1.0,24.29036,16.9862,False,DR8,530781,DR8-2837p402-5033,1,True,283.619631436471,40.3162008554537,0.54342,2837p402,283.6196056669463,40.316337272867564,0.53864765,SB26,179.53694,0.3690244,283.6194583471782,40.31653379127378,11.641381,11.157478,8.614751,9.8993,1.6142721,3.685578,5.5844307,7.701021,9.878292,11.927793,13.400182,14.697571,16.15943,21.37251,20.373438,19.779512,19.920574,19.005274,18.41066,19.314016,18.435898,17.843414,18.922651,18.076838,17.494484,18.664383,17.833265,17.25879,18.496494,17.682926,17.106598,18.404135,17.605488,17.02742,18.332602,17.555883,16.971588,18.299273,17.52499,16.931496,0.04933311,0.065890566,0.09742977,0.10495926,0.1614855,0.27832195,0.15179653,0.13904023,0.20484819,0.041657884,0.06669066,0.06785251,0.038538612,0.049561314,0.05101378,0.03635901,0.041818976,0.042013034,0.03367074,0.03763669,0.03705858,0.03122176,0.03420294,0.033616345,0.029198052,0.031941395,0.031132452,0.028080799,0.030865192,0.029955743,0.02722575,0.030135926,0.029090546,0.026788738,0.02959205,0.028436601,17.799881,1.3952494,0.83855844,1.4560318,0.5604667,17.200954,1.0272083,0.8301853,1.7677885,0.4513309,16.501709,1.3551984,0.73172045,1.4457989,0.2076136,0,-1.0,-1.0,-1.0,-1
5001827,SGA-2020 5001827,DR8-3429p190-2654,-1,343.06599510416885,19.007842544240166,COMP,149.72781,0

# SN1a Distances

Note that the `ASAS-SN SNIa` table on the EDD does *not* have any distance moduli and is therefore not included below.

We are also skipping Tonry's data (https://ui.adsabs.harvard.edu/abs/2003ApJ...594....1T/abstract, Table 8) because the distances are in velocity units, which makes me (KAD) think that they are not actually distance moduli.

In [17]:
sn_dist = glob('cache/SN1A_dist/*')
sn_dist[0]

'cache/SN1A_dist/EDDtable_Constitution-SNIa.txt'

In [18]:
len(sn_dist)

12

In [19]:
# sn_dist[0].split('/')[-1].strip('.txt').strip('EDDtable')
sn_dist[0].split('/')[-1].strip('.txt').split('_')[-1]

'Constitution-SNIa'

In [20]:
PGC_ids = []
DM1 = []
e_DM1 = []
DM2 = []
e_DM2 = []
DM3 = []
e_DM3 = []
DM4 = []
e_DM4 = []
DM5 = []
e_DM5 = []
DM6 = []
e_DM6 = []
DM7 = []
e_DM7 = []
dist = []
e_dist = []
catalog = []

for sfile in sn_dist:
    
    if sfile == 'cache/SN1A_dist/EDDtable_Tonry-SNIa.txt':
        continue
    print(sfile)
    
    # Extract catalog name from file name
    #name = sfile.split('/')[-1].strip('.txt').strip('EDDtable')
    name = sfile.split('/')[-1].strip('.txt').split('_')[-1]
    
    # Read data from file
    tab = Table.read(sfile, format='csv', header_start=1, data_start=5)
    
    # Rename pgc column name to match SGA
    tab.rename_column('pgc', 'PGC')
    
    jtab = join(SGA, tab, keys='PGC', join_type='inner')
    print(tab.colnames)
    
    # Append catalog name to list of catalogs
    catalog.append(np.full(np.shape(jtab['PGC'].flatten()), name))
    
    # Append PGC numbers to list of PGC numbers
    PGC_ids.append(jtab['PGC'].flatten())
    
    # Append distance moduli and uncertainties to appropriate lists
    try:
        ## SALT
        DM1.append(np.array(jtab['S_mu0']).flatten())
        # print(jtab['S_mu0'].flatten())
        e_DM1.append(np.array(jtab['S_emu0']).flatten())
        # print(len(np.array(jtab['S_mu0']).flatten()))
        # print(len(np.array(jtab['S_emu0']).flatten()))
    except KeyError:
        try:
            DM1.append(np.array(jtab['mu0']).flatten())
            e_DM1.append(np.array(jtab['emu0']).flatten())
            # print(len(np.array(jtab['mu0']).flatten()))
            # print(len(np.array(jtab['emu0']).flatten()))
        except KeyError:
            try:
                DM1.append(np.array(jtab['muCV']).flatten())
                e_DM1.append(np.array(jtab['emu']).flatten())
                # print(len(np.array(jtab['muCV']).flatten()))
                # print(len(np.array(jtab['emu']).flatten()))
            except KeyError:
                try:
                    DM1.append(np.array(jtab['mu']).flatten())
                    e_DM1.append(np.array(jtab['emu']).flatten())
                    # print(len(np.array(jtab['mu']).flatten()))
                    # print(len(np.array(jtab['emu']).flatten()))
                except KeyError:
                    try:
                        ## mu0U: Union2 (2010) Distance Modulus assuming H0=74.2 km/s/Mpc
                        DM1.append(np.array(jtab['mu0U']).flatten())
                        e_DM1.append(np.array(jtab['emu0U']).flatten())
                        # print(len(np.array(jtab['mu0U']).flatten()))
                        # print(len(np.array(jtab['emu0U']).flatten()))
                    except KeyError:
                        try:
                            DM1.append(np.array(jtab['DM']).flatten())
                            e_DM1.append(np.array(jtab['eDM']).flatten())
                            # print(len(np.array(jtab['DM']).flatten()))
                            # print(len(np.array(jtab['eDM']).flatten()))
                        except KeyError:
                            try:
                                ## DM5c: Distance modulus, all contributing sources, Cosmicflows-2 zero point
                                DM1.append(np.array(jtab['DM5c']).flatten())
                                e_DM1.append(np.full(np.shape(np.array(jtab['PGC']).flatten()), 0))
                                # print(len(np.array(jtab['DM5c']).flatten()))
                                # print(len(np.array(jtab['eDM5c']).flatten()))
                            except KeyError:
                                print('DM and eDM not found')
    
    # Repeat for second distance moduli (when available)
    try:
        ## S2_mu0: SALT2
        DM2.append(np.array(jtab['S2_mu0']).flatten())
        e_DM2.append(np.array(jtab['S2_emu0']).flatten())
        # print(len(np.array(jtab['S2_mu0']).flatten()))
        # print(len(np.array(jtab['S2_emu0']).flatten()))
    except KeyError:
        try:
            ## mu0J: Jha et al. (2007) Distance Modulus assuming H0=65 km/s/Mpc 
            DM2.append(np.array(jtab['mu0J']).flatten())
            e_DM2.append(np.array(jtab['emu0J']).flatten())
            # print(len(np.array(jtab['mu0J']).flatten()))
            # print(len(np.array(jtab['emu0J']).flatten()))
        except:
            try:
                ## DM5u: Distance modulus, all contributing sources, Union2 zero point
                DM2.append(np.array(jtab['DM5u']).flatten())
                e_DM2.append(np.array(jtab['DM5u']).flatten())
                # print(len(np.array(jtab['DM5u']).flatten()))
                # print(len(np.array(jtab['DM5u']).flatten()))
            except KeyError:
                DM2.append(np.full(np.shape(np.array(jtab['PGC']).flatten()), -1))
                e_DM2.append(np.full(np.shape(np.array(jtab['PGC']).flatten()), -1))
    
    # And for third distance moduli (when available)
    try:
        ## M31_mu0
        DM3.append(np.array(jtab['M31_mu0']).flatten())
        e_DM3.append(np.array(jtab['M31_emu0']).flatten())
        # print(len(np.array(jtab['M31_mu0']).flatten()))
        # print(len(np.array(jtab['M31_emu0']).flatten()))
    except KeyError:
        try:
            ## DMu: Distance modulus from Union2 (Amanullah et al. 2010), Union2 zero point 
            DM3.append(np.array(jtab['DMu']).flatten())
            e_DM3.append(np.array(jtab['eDMu']).flatten())
            # print(len(np.array(jtab['DMu']).flatten()))
            # print(len(np.array(jtab['DMu']).flatten()))
        except KeyError:
            DM3.append(np.full(np.shape(np.array(jtab['PGC']).flatten()), -1))
            e_DM3.append(np.full(np.shape(np.array(jtab['PGC']).flatten()), -1))
    
    # And for the fourth distance moduli (when available)
    try:
        ## M17_mu0
        DM4.append(np.array(jtab['M17_mu0']).flatten())
        e_DM4.append(np.array(jtab['M17_emu0']).flatten())
    except KeyError:
        try:
            ## DMj: Distance modulus from Jha et al. (2007), Union2 zero point
            DM4.append(np.array(jtab['DMj']).flatten())
            e_DM4.append(np.array(jtab['eDMj']).flatten())
        except KeyError:
            DM4.append(np.full(np.shape(np.array(jtab['PGC']).flatten()), -1))
            e_DM4.append(np.full(np.shape(np.array(jtab['PGC']).flatten()), -1))
    
    # And for the fifth distance moduli (when available)
    try:
        ## DMp: Distance modulus from Prieto et al. (2006), Union2 zero point
        DM5.append(np.array(jtab['DMp']).flatten())
        e_DM5.append(np.array(jtab['eDMp']).flatten())
    except KeyError:
        # print(np.shape(flatten_array(jtab['PGC'])))
        # print(np.shape(np.array(np.array(jtab['PGC'])).flatten()))
        # l = np.array(np.array(jtab['PGC'])).flatten()
        DM5.append(np.full(np.shape(np.array(jtab['PGC']).flatten()), -1))
        e_DM5.append(np.full(np.shape(np.array(jtab['PGC']).flatten()), -1))
    
    # And for the sixth and seventh distance moduli (when available)
    try:
        ## DMh: Distance modulus from Hicken et al. (2009), Union2 zero point
        ## DMf: Distance modulus from Folatelli et al. (2010)
        DM6.append(np.array(jtab['DMh']).flatten())
        DM7.append(np.array(jtab['DMf']).flatten())
        e_DM6.append(np.full(np.shape(np.array(jtab['PGC']).flatten()), 0))
        e_DM7.append(np.full(np.shape(np.array(jtab['PGC']).flatten()), 0))
    except KeyError:
        DM6.append(np.full(np.shape(np.array(jtab['PGC']).flatten()), -1))
        DM7.append(np.full(np.shape(np.array(jtab['PGC']).flatten()), -1))
        e_DM6.append(np.full(np.shape(np.array(jtab['PGC']).flatten()), -1))
        e_DM7.append(np.full(np.shape(np.array(jtab['PGC']).flatten()), -1))
    
    print()

cache/SN1A_dist/EDDtable_Constitution-SNIa.txt
['PGC', 'SNIa', 'zcmb', 'ez', 'S_Bmag', 'S_eB', 'S_s', 'S_es', 'S_c', 'S_ec', 'S_mu0', 'S_emu0', 'S2_Bmag', 'S2_eB', 'S2_s', 'S2_es', 'S2_c', 'S2_ec', 'S2_mu0', 'S2_emu0', 'M31_Del', 'M31_eD', 'M31_AV', 'M31_eAV', 'M31_mu0', 'M31_emu0', 'M17_Del', 'M17_eD', 'M17_AV', 'M17_eAV', 'M17_mu0', 'M17_emu0']

cache/SN1A_dist/EDDtable_SNIa-calibration.txt
['PGC', 'SNIa', 'objname', 'J2000', 'Glon', 'Glat', 'SGL', 'SGB', 'Typ', 'logD25', 'logR25', 'BT', 'IT', 'Ag', 'Vhel', 'Vcmb', 'VcmbSN', 'mu0U', 'emu0U', 'Cuts', 'mu0J', 'emu0J', 'NotesJ', 'Notes']

cache/SN1A_dist/EDDtable_CSP1-SNIa.txt
['PGC', 'SNIa', 'zhel', 'zcmb', 'E(B-V)', 'DelM15', 'eM15', 'mu0', 'emu0', 'Q']

cache/SN1A_dist/EDDtable_Prieto-SNIa.txt
['PGC', 'SNIa', 'z', 'mu0', 'emu0', 'E(B-V)', 'eE(B-V)', 'DelM15', 'eM15']

cache/SN1A_dist/EDDtable_5-Sources-SNIa.txt
['PGC', 'SNIa', 'l', 'b', 'SGL', 'SGB', 'Vhel', 'Vcmb', 'z', 'logz', 'Ns', 'DM5c', 'D5c', 'H5c', 'DM5u', 'D5u', 'H5u', 'DMu'

In [21]:
PGC_ids_flat = flatten_list(PGC_ids)
DM1_flat = flatten_list(DM1)
e_DM1_flat = flatten_list(e_DM1)
DM2_flat = flatten_list(DM2)
e_DM2_flat = flatten_list(e_DM2)
DM3_flat = flatten_list(DM3)
e_DM3_flat = flatten_list(e_DM3)
DM4_flat = flatten_list(DM4)
e_DM4_flat = flatten_list(e_DM4)
DM5_flat = flatten_list(DM5)
e_DM5_flat = flatten_list(e_DM5)
DM6_flat = flatten_list(DM6)
e_DM6_flat = flatten_list(e_DM6)
DM7_flat = flatten_list(DM7)
e_DM7_flat = flatten_list(e_DM7)
catalog_flat = flatten_list(catalog)

In [22]:
len(e_DM1)

11

In [23]:
metadata = {
            'DM1_SN': 'SALT data for Constitution_SNIa; Union2 (2010) Distance Modulus assuming H0=74.2 km/s/Mpc for SNIa_calibration; Distance modulus, all contributing sources, Cosmicflows-2 zero point for 5_Sources_SNIa',
            'DM2_SN': 'SALT2 data for Constitution_SNIa; Jha et al. (2007) Distance Modulus assuming H0=65 km/s/Mpc for SNIa_calibration; Distance modulus, all contributing sources, Union2 zero point for 5_Sources_SNIa',
            'DM3_SN': 'MLCS31 data for Constitution_SNIa; Distance modulus from Union2 (Amanullah et al. 2010), Union2 zero point for 5_Sources_SNIa',
            'DM4_SN': 'MLCS17 data for Constitution_SNIa; Distance modulus from Jha et al. (2007), Union2 zero point for 5_Sources_SNIa',
            'DM5_SN': 'Distance modulus from Prieto et al. (2006), Union2 zero point for 5_Sources_SNIa',
            'DM6_SN': 'Distance modulus from Hicken et al. (2009), Union2 zero point for 5_Sources_SNIa (no error given)',
            'DM7_SN': 'Distance modulus from Folatelli et al. (2010) for 5_Sources_SNIa (no error given)'
            }

In [24]:
tab = Table([PGC_ids_flat, DM1_flat, e_DM1_flat, DM2_flat, e_DM2_flat, DM3_flat, e_DM3_flat, DM4_flat, e_DM4_flat,DM5_flat, e_DM5_flat,DM6_flat, e_DM6_flat,DM7_flat, e_DM7_flat, catalog_flat],
             names = ['PGC', 'DM1_SN', 'e_DM1_SN',  'DM2_SN', 'e_DM2_SN', 'DM3_SN', 'e_DM3_SN', 'DM4_SN', 'e_DM4_SN', 'DM5_SN', 'e_DM5_SN', 'DM6_SN', 'e_DM6_SN', 'DM7_SN', 'e_DM7_SN','SN_Catalog'],
             meta = metadata)

tab[:5]

PGC,DM1_SN,e_DM1_SN,DM2_SN,e_DM2_SN,DM3_SN,e_DM3_SN,DM4_SN,e_DM4_SN,DM5_SN,e_DM5_SN,DM6_SN,e_DM6_SN,DM7_SN,e_DM7_SN,SN_Catalog
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,int64,str25
250,35.272,0.185,35.241,0.204,35.282,0.192,35.279,0.177,-1.0,-1.0,-1.0,-1,-1.0,-1,Constitution-SNIa
415,35.358,0.185,35.383,0.212,35.248,0.188,35.282,0.187,-1.0,-1.0,-1.0,-1,-1.0,-1,Constitution-SNIa
515,0.0,0.0,34.574,0.245,34.037,0.216,34.33,0.212,-1.0,-1.0,-1.0,-1,-1.0,-1,Constitution-SNIa
2332,34.962,0.202,34.91,0.227,34.974,0.196,34.858,0.202,-1.0,-1.0,-1.0,-1,-1.0,-1,Constitution-SNIa
2501,36.685,0.157,36.769,0.184,36.763,0.17,36.776,0.158,-1.0,-1.0,-1.0,-1,-1.0,-1,Constitution-SNIa


In [25]:
maintab2 = join(maintab, tab, keys='PGC', join_type='left')

maintab2[:5]

SGA_ID,SGA_GALAXY,GALAXY,PGC,RA_LEDA,DEC_LEDA,MORPHTYPE,PA_LEDA,D25_LEDA,BA_LEDA,Z_LEDA,SB_D25_LEDA,MAG_LEDA,BYHAND,REF,GROUP_ID,GROUP_NAME,GROUP_MULT,GROUP_PRIMARY,GROUP_RA,GROUP_DEC,GROUP_DIAMETER,BRICKNAME,RA,DEC,D26,D26_REF,PA,BA,RA_MOMENT,DEC_MOMENT,SMA_MOMENT,G_SMA50,R_SMA50,Z_SMA50,SMA_SB22,SMA_SB22.5,SMA_SB23,SMA_SB23.5,SMA_SB24,SMA_SB24.5,SMA_SB25,SMA_SB25.5,SMA_SB26,G_MAG_SB22,R_MAG_SB22,Z_MAG_SB22,G_MAG_SB22.5,R_MAG_SB22.5,Z_MAG_SB22.5,G_MAG_SB23,R_MAG_SB23,Z_MAG_SB23,G_MAG_SB23.5,R_MAG_SB23.5,Z_MAG_SB23.5,G_MAG_SB24,R_MAG_SB24,Z_MAG_SB24,G_MAG_SB24.5,R_MAG_SB24.5,Z_MAG_SB24.5,G_MAG_SB25,R_MAG_SB25,Z_MAG_SB25,G_MAG_SB25.5,R_MAG_SB25.5,Z_MAG_SB25.5,G_MAG_SB26,R_MAG_SB26,Z_MAG_SB26,SMA_SB22_ERR,SMA_SB22.5_ERR,SMA_SB23_ERR,SMA_SB23.5_ERR,SMA_SB24_ERR,SMA_SB24.5_ERR,SMA_SB25_ERR,SMA_SB25.5_ERR,SMA_SB26_ERR,G_MAG_SB22_ERR,R_MAG_SB22_ERR,Z_MAG_SB22_ERR,G_MAG_SB22.5_ERR,R_MAG_SB22.5_ERR,Z_MAG_SB22.5_ERR,G_MAG_SB23_ERR,R_MAG_SB23_ERR,Z_MAG_SB23_ERR,G_MAG_SB23.5_ERR,R_MAG_SB23.5_ERR,Z_MAG_SB23.5_ERR,G_MAG_SB24_ERR,R_MAG_SB24_ERR,Z_MAG_SB24_ERR,G_MAG_SB24.5_ERR,R_MAG_SB24.5_ERR,Z_MAG_SB24.5_ERR,G_MAG_SB25_ERR,R_MAG_SB25_ERR,Z_MAG_SB25_ERR,G_MAG_SB25.5_ERR,R_MAG_SB25.5_ERR,Z_MAG_SB25.5_ERR,G_MAG_SB26_ERR,R_MAG_SB26_ERR,Z_MAG_SB26_ERR,G_COG_PARAMS_MTOT,G_COG_PARAMS_M0,G_COG_PARAMS_ALPHA1,G_COG_PARAMS_ALPHA2,G_COG_PARAMS_CHI2,R_COG_PARAMS_MTOT,R_COG_PARAMS_M0,R_COG_PARAMS_ALPHA1,R_COG_PARAMS_ALPHA2,R_COG_PARAMS_CHI2,Z_COG_PARAMS_MTOT,Z_COG_PARAMS_M0,Z_COG_PARAMS_ALPHA1,Z_COG_PARAMS_ALPHA2,Z_COG_PARAMS_CHI2,ELLIPSEBIT,DM_Stellar,e_DM_lo_Stellar,e_DM_hi_Stellar,Stellar_Catalog,DM1_SN,e_DM1_SN,DM2_SN,e_DM2_SN,DM3_SN,e_DM3_SN,DM4_SN,e_DM4_SN,DM5_SN,e_DM5_SN,DM6_SN,e_DM6_SN,DM7_SN,e_DM7_SN,SN_Catalog
int64,bytes16,bytes29,int64,float64,float64,bytes21,float32,float32,float32,float32,float32,float32,bool,bytes13,int64,bytes35,int16,bool,float64,float64,float32,bytes8,float64,float64,float32,bytes4,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,float64,float64,float64,str14,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,int64,str25
5002480,SGA-2020 5002480,DR8-2837p402-5033,-1,283.619631436471,40.3162008554537,DEV,178.80934,0.54342,0.3166,-1.0,24.29036,16.9862,False,DR8,530781,DR8-2837p402-5033,1,True,283.619631436471,40.3162008554537,0.54342,2837p402,283.6196056669463,40.316337272867564,0.53864765,SB26,179.53694,0.3690244,283.6194583471782,40.31653379127378,11.641381,11.157478,8.614751,9.8993,1.6142721,3.685578,5.5844307,7.701021,9.878292,11.927793,13.400182,14.697571,16.15943,21.37251,20.373438,19.779512,19.920574,19.005274,18.41066,19.314016,18.435898,17.843414,18.922651,18.076838,17.494484,18.664383,17.833265,17.25879,18.496494,17.682926,17.106598,18.404135,17.605488,17.02742,18.332602,17.555883,16.971588,18.299273,17.52499,16.931496,0.04933311,0.065890566,0.09742977,0.10495926,0.1614855,0.27832195,0.15179653,0.13904023,0.20484819,0.041657884,0.06669066,0.06785251,0.038538612,0.049561314,0.05101378,0.03635901,0.041818976,0.042013034,0.03367074,0.03763669,0.03705858,0.03122176,0.03420294,0.033616345,0.029198052,0.031941395,0.031132452,0.028080799,0.030865192,0.029955743,0.02722575,0.030135926,0.029090546,0.026788738,0.02959205,0.028436601,17.799881,1.3952494,0.83855844,1.45

In [26]:
'''
maintab2['DM1_SN'].fill_value = -1
maintab2['e_DM1_SN'].fill_value = -1
maintab2['DM2_SN'].fill_value = -1
maintab2['e_DM2_SN'].fill_value = -1
maintab2['DM3_SN'].fill_value = -1
maintab2['e_DM3_SN'].fill_value = -1
maintab2['DM4_SN'].fill_value = -1
maintab2['e_DM4_SN'].fill_value = -1
maintab2['DM5_SN'].fill_value = -1
maintab2['e_DM5_SN'].fill_value = -1
maintab2['DM6_SN'].fill_value = -1
maintab2['e_DM6_SN'].fill_value = -1
maintab2['DM7_SN'].fill_value = -1
maintab2['e_DM7_SN'].fill_value = -1
maintab2['SN_Catalog'].fill_value = -1
''';
maintab2 = maintab2.filled(-1)

In [27]:
maintab2

SGA_ID,SGA_GALAXY,GALAXY,PGC,RA_LEDA,DEC_LEDA,MORPHTYPE,PA_LEDA,D25_LEDA,BA_LEDA,Z_LEDA,SB_D25_LEDA,MAG_LEDA,BYHAND,REF,GROUP_ID,GROUP_NAME,GROUP_MULT,GROUP_PRIMARY,GROUP_RA,GROUP_DEC,GROUP_DIAMETER,BRICKNAME,RA,DEC,D26,D26_REF,PA,BA,RA_MOMENT,DEC_MOMENT,SMA_MOMENT,G_SMA50,R_SMA50,Z_SMA50,SMA_SB22,SMA_SB22.5,SMA_SB23,SMA_SB23.5,SMA_SB24,SMA_SB24.5,SMA_SB25,SMA_SB25.5,SMA_SB26,G_MAG_SB22,R_MAG_SB22,Z_MAG_SB22,G_MAG_SB22.5,R_MAG_SB22.5,Z_MAG_SB22.5,G_MAG_SB23,R_MAG_SB23,Z_MAG_SB23,G_MAG_SB23.5,R_MAG_SB23.5,Z_MAG_SB23.5,G_MAG_SB24,R_MAG_SB24,Z_MAG_SB24,G_MAG_SB24.5,R_MAG_SB24.5,Z_MAG_SB24.5,G_MAG_SB25,R_MAG_SB25,Z_MAG_SB25,G_MAG_SB25.5,R_MAG_SB25.5,Z_MAG_SB25.5,G_MAG_SB26,R_MAG_SB26,Z_MAG_SB26,SMA_SB22_ERR,SMA_SB22.5_ERR,SMA_SB23_ERR,SMA_SB23.5_ERR,SMA_SB24_ERR,SMA_SB24.5_ERR,SMA_SB25_ERR,SMA_SB25.5_ERR,SMA_SB26_ERR,G_MAG_SB22_ERR,R_MAG_SB22_ERR,Z_MAG_SB22_ERR,G_MAG_SB22.5_ERR,R_MAG_SB22.5_ERR,Z_MAG_SB22.5_ERR,G_MAG_SB23_ERR,R_MAG_SB23_ERR,Z_MAG_SB23_ERR,G_MAG_SB23.5_ERR,R_MAG_SB23.5_ERR,Z_MAG_SB23.5_ERR,G_MAG_SB24_ERR,R_MAG_SB24_ERR,Z_MAG_SB24_ERR,G_MAG_SB24.5_ERR,R_MAG_SB24.5_ERR,Z_MAG_SB24.5_ERR,G_MAG_SB25_ERR,R_MAG_SB25_ERR,Z_MAG_SB25_ERR,G_MAG_SB25.5_ERR,R_MAG_SB25.5_ERR,Z_MAG_SB25.5_ERR,G_MAG_SB26_ERR,R_MAG_SB26_ERR,Z_MAG_SB26_ERR,G_COG_PARAMS_MTOT,G_COG_PARAMS_M0,G_COG_PARAMS_ALPHA1,G_COG_PARAMS_ALPHA2,G_COG_PARAMS_CHI2,R_COG_PARAMS_MTOT,R_COG_PARAMS_M0,R_COG_PARAMS_ALPHA1,R_COG_PARAMS_ALPHA2,R_COG_PARAMS_CHI2,Z_COG_PARAMS_MTOT,Z_COG_PARAMS_M0,Z_COG_PARAMS_ALPHA1,Z_COG_PARAMS_ALPHA2,Z_COG_PARAMS_CHI2,ELLIPSEBIT,DM_Stellar,e_DM_lo_Stellar,e_DM_hi_Stellar,Stellar_Catalog,DM1_SN,e_DM1_SN,DM2_SN,e_DM2_SN,DM3_SN,e_DM3_SN,DM4_SN,e_DM4_SN,DM5_SN,e_DM5_SN,DM6_SN,e_DM6_SN,DM7_SN,e_DM7_SN,SN_Catalog
int64,bytes16,bytes29,int64,float64,float64,bytes21,float32,float32,float32,float32,float32,float32,bool,bytes13,int64,bytes35,int16,bool,float64,float64,float32,bytes8,float64,float64,float32,bytes4,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,float64,float64,float64,str14,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,int64,str25
5002480,SGA-2020 5002480,DR8-2837p402-5033,-1,283.619631436471,40.3162008554537,DEV,178.80934,0.54342,0.3166,-1.0,24.29036,16.9862,False,DR8,530781,DR8-2837p402-5033,1,True,283.619631436471,40.3162008554537,0.54342,2837p402,283.6196056669463,40.316337272867564,0.53864765,SB26,179.53694,0.3690244,283.6194583471782,40.31653379127378,11.641381,11.157478,8.614751,9.8993,1.6142721,3.685578,5.5844307,7.701021,9.878292,11.927793,13.400182,14.697571,16.15943,21.37251,20.373438,19.779512,19.920574,19.005274,18.41066,19.314016,18.435898,17.843414,18.922651,18.076838,17.494484,18.664383,17.833265,17.25879,18.496494,17.682926,17.106598,18.404135,17.605488,17.02742,18.332602,17.555883,16.971588,18.299273,17.52499,16.931496,0.04933311,0.065890566,0.09742977,0.10495926,0.1614855,0.27832195,0.15179653,0.13904023,0.20484819,0.041657884,0.06669066,0.06785251,0.038538612,0.049561314,0.05101378,0.03635901,0.041818976,0.042013034,0.03367074,0.03763669,0.03705858,0.03122176,0.03420294,0.033616345,0.029198052,0.031941395,0.031132452,0.028080799,0.030865192,0.029955743,0.02722575,0.030135926,0.029090546,0.026788738,0.02959205,0.028436601,17.799881,1.3952494,0.83855844,1.45

In [28]:
maintab2.write('SGA_distances.fits', format='fits')

# Combining with DESI Rosettes

In [48]:
centers = Table.read('cache/tiles-sv3.ecsv', format = 'ascii.ecsv')
centers[0]['TILEID']

1

In [77]:
# m = (maintab2['DM_Stellar'] != -1) or (maintain2['DM1_SN'] != -1)
disttab = maintab2[(maintab2['DM_Stellar'] != -1) | (maintab2['DM1_SN'] != -1)]
disttab

SGA_ID,SGA_GALAXY,GALAXY,PGC,RA_LEDA,DEC_LEDA,MORPHTYPE,PA_LEDA,D25_LEDA,BA_LEDA,Z_LEDA,SB_D25_LEDA,MAG_LEDA,BYHAND,REF,GROUP_ID,GROUP_NAME,GROUP_MULT,GROUP_PRIMARY,GROUP_RA,GROUP_DEC,GROUP_DIAMETER,BRICKNAME,RA,DEC,D26,D26_REF,PA,BA,RA_MOMENT,DEC_MOMENT,SMA_MOMENT,G_SMA50,R_SMA50,Z_SMA50,SMA_SB22,SMA_SB22.5,SMA_SB23,SMA_SB23.5,SMA_SB24,SMA_SB24.5,SMA_SB25,SMA_SB25.5,SMA_SB26,G_MAG_SB22,R_MAG_SB22,Z_MAG_SB22,G_MAG_SB22.5,R_MAG_SB22.5,Z_MAG_SB22.5,G_MAG_SB23,R_MAG_SB23,Z_MAG_SB23,G_MAG_SB23.5,R_MAG_SB23.5,Z_MAG_SB23.5,G_MAG_SB24,R_MAG_SB24,Z_MAG_SB24,G_MAG_SB24.5,R_MAG_SB24.5,Z_MAG_SB24.5,G_MAG_SB25,R_MAG_SB25,Z_MAG_SB25,G_MAG_SB25.5,R_MAG_SB25.5,Z_MAG_SB25.5,G_MAG_SB26,R_MAG_SB26,Z_MAG_SB26,SMA_SB22_ERR,SMA_SB22.5_ERR,SMA_SB23_ERR,SMA_SB23.5_ERR,SMA_SB24_ERR,SMA_SB24.5_ERR,SMA_SB25_ERR,SMA_SB25.5_ERR,SMA_SB26_ERR,G_MAG_SB22_ERR,R_MAG_SB22_ERR,Z_MAG_SB22_ERR,G_MAG_SB22.5_ERR,R_MAG_SB22.5_ERR,Z_MAG_SB22.5_ERR,G_MAG_SB23_ERR,R_MAG_SB23_ERR,Z_MAG_SB23_ERR,G_MAG_SB23.5_ERR,R_MAG_SB23.5_ERR,Z_MAG_SB23.5_ERR,G_MAG_SB24_ERR,R_MAG_SB24_ERR,Z_MAG_SB24_ERR,G_MAG_SB24.5_ERR,R_MAG_SB24.5_ERR,Z_MAG_SB24.5_ERR,G_MAG_SB25_ERR,R_MAG_SB25_ERR,Z_MAG_SB25_ERR,G_MAG_SB25.5_ERR,R_MAG_SB25.5_ERR,Z_MAG_SB25.5_ERR,G_MAG_SB26_ERR,R_MAG_SB26_ERR,Z_MAG_SB26_ERR,G_COG_PARAMS_MTOT,G_COG_PARAMS_M0,G_COG_PARAMS_ALPHA1,G_COG_PARAMS_ALPHA2,G_COG_PARAMS_CHI2,R_COG_PARAMS_MTOT,R_COG_PARAMS_M0,R_COG_PARAMS_ALPHA1,R_COG_PARAMS_ALPHA2,R_COG_PARAMS_CHI2,Z_COG_PARAMS_MTOT,Z_COG_PARAMS_M0,Z_COG_PARAMS_ALPHA1,Z_COG_PARAMS_ALPHA2,Z_COG_PARAMS_CHI2,ELLIPSEBIT,DM_Stellar,e_DM_lo_Stellar,e_DM_hi_Stellar,Stellar_Catalog,DM1_SN,e_DM1_SN,DM2_SN,e_DM2_SN,DM3_SN,e_DM3_SN,DM4_SN,e_DM4_SN,DM5_SN,e_DM5_SN,DM6_SN,e_DM6_SN,DM7_SN,e_DM7_SN,SN_Catalog
int64,bytes16,bytes29,int64,float64,float64,bytes21,float32,float32,float32,float32,float32,float32,bool,bytes13,int64,bytes35,int16,bool,float64,float64,float32,bytes8,float64,float64,float32,bytes4,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,float64,float64,float64,str16,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,int64,str24
207638,SGA-2020 207638,NGC7814,218,0.812559,16.1456194,Sab,134.53,4.365158,0.4305266,0.0035124298,23.421482,11.593,False,LEDA-20181114,73938,NGC7814,1,True,0.812559,16.1456194,4.365158,0009p162,0.8124274854816511,16.1455169162983,11.15105,SB26,134.2024,0.4849028,0.8120762037951955,16.14534837690191,200.04625,82.82556,78.24203,72.30593,94.85372,118.66765,144.39467,172.24834,202.94002,234.82027,267.05215,302.1296,334.5315,11.391324,10.4479065,9.623682,11.228865,10.295919,9.4892435,11.108571,10.186071,9.391858,11.016354,10.10016,9.319667,10.945187,10.035046,9.264916,10.894269,9.989213,9.225079,10.856926,9.954834,9.195862,10.830645,9.929288,9.171985,10.814159,9.9131775,9.156266,0.21526095,0.23602074,0.34232575,0.3225715,0.26907054,0.4221382,0.53733534,0.39212316,0.53470826,0.0020625296,0.0022948585,0.002650695,0.0018137891,0.0020279833,0.0023673968,0.0016452036,0.0018506112,0.0021778245,0.0015238288,0.0017204144,0.0020450433,0.0014345231,0.0016259744,0.0019482705,0.0013731329,0.001561975,0.0018802665,0.001329639,0.0015156246,0.0018319507,0.0012991042,0.0014813977,0.0017928901,0.001279939,0.0014598139,0.0017673312,10.7426405,0.3634739,1342.3901,2.4910772

In [81]:
sep_max = 1.45 * u.deg
sep_min = 0.15 * u.deg
SGA_match = []
with open("rosette_match_output_dist.txt", "a") as f:
    for i in range(len(centers)):
        print('TILE ID: ', centers[i]['TILEID'], file = f)
        ra, dec = centers[i]['RA'] * u.deg, centers[i]['DEC'] * u.deg
        center = SkyCoord(ra, dec)
        for j in range(len(disttab)):
            ra2, dec2 = disttab[j]['RA'] * u.deg, disttab[j]['DEC'] * u.deg
            target = SkyCoord(ra2, dec2)
            d2d = center.separation(target)
            msk1 = d2d < sep_max
            msk2 = d2d > sep_min
            if msk1 == True and msk2 == True:
                # print('Match found')
                SGA_match.append(disttab[j]['SGA_ID'])
                print('Match found:', disttab[j]['SGA_ID'], file = f)
            # print()
            # break
        print(file = f)
        # break
# f.close()

In [80]:
f.close()

In [84]:
(np.unique(SGA_match))

array([  49524,  135945,  294387,  464075,  474506,  497874,  630029,
        662902,  772284, 1065602, 1271764, 1351452, 1385252, 1413430,
       2000090])

In [85]:
disttab[disttab['SGA_ID'] == 2000090]

SGA_ID,SGA_GALAXY,GALAXY,PGC,RA_LEDA,DEC_LEDA,MORPHTYPE,PA_LEDA,D25_LEDA,BA_LEDA,Z_LEDA,SB_D25_LEDA,MAG_LEDA,BYHAND,REF,GROUP_ID,GROUP_NAME,GROUP_MULT,GROUP_PRIMARY,GROUP_RA,GROUP_DEC,GROUP_DIAMETER,BRICKNAME,RA,DEC,D26,D26_REF,PA,BA,RA_MOMENT,DEC_MOMENT,SMA_MOMENT,G_SMA50,R_SMA50,Z_SMA50,SMA_SB22,SMA_SB22.5,SMA_SB23,SMA_SB23.5,SMA_SB24,SMA_SB24.5,SMA_SB25,SMA_SB25.5,SMA_SB26,G_MAG_SB22,R_MAG_SB22,Z_MAG_SB22,G_MAG_SB22.5,R_MAG_SB22.5,Z_MAG_SB22.5,G_MAG_SB23,R_MAG_SB23,Z_MAG_SB23,G_MAG_SB23.5,R_MAG_SB23.5,Z_MAG_SB23.5,G_MAG_SB24,R_MAG_SB24,Z_MAG_SB24,G_MAG_SB24.5,R_MAG_SB24.5,Z_MAG_SB24.5,G_MAG_SB25,R_MAG_SB25,Z_MAG_SB25,G_MAG_SB25.5,R_MAG_SB25.5,Z_MAG_SB25.5,G_MAG_SB26,R_MAG_SB26,Z_MAG_SB26,SMA_SB22_ERR,SMA_SB22.5_ERR,SMA_SB23_ERR,SMA_SB23.5_ERR,SMA_SB24_ERR,SMA_SB24.5_ERR,SMA_SB25_ERR,SMA_SB25.5_ERR,SMA_SB26_ERR,G_MAG_SB22_ERR,R_MAG_SB22_ERR,Z_MAG_SB22_ERR,G_MAG_SB22.5_ERR,R_MAG_SB22.5_ERR,Z_MAG_SB22.5_ERR,G_MAG_SB23_ERR,R_MAG_SB23_ERR,Z_MAG_SB23_ERR,G_MAG_SB23.5_ERR,R_MAG_SB23.5_ERR,Z_MAG_SB23.5_ERR,G_MAG_SB24_ERR,R_MAG_SB24_ERR,Z_MAG_SB24_ERR,G_MAG_SB24.5_ERR,R_MAG_SB24.5_ERR,Z_MAG_SB24.5_ERR,G_MAG_SB25_ERR,R_MAG_SB25_ERR,Z_MAG_SB25_ERR,G_MAG_SB25.5_ERR,R_MAG_SB25.5_ERR,Z_MAG_SB25.5_ERR,G_MAG_SB26_ERR,R_MAG_SB26_ERR,Z_MAG_SB26_ERR,G_COG_PARAMS_MTOT,G_COG_PARAMS_M0,G_COG_PARAMS_ALPHA1,G_COG_PARAMS_ALPHA2,G_COG_PARAMS_CHI2,R_COG_PARAMS_MTOT,R_COG_PARAMS_M0,R_COG_PARAMS_ALPHA1,R_COG_PARAMS_ALPHA2,R_COG_PARAMS_CHI2,Z_COG_PARAMS_MTOT,Z_COG_PARAMS_M0,Z_COG_PARAMS_ALPHA1,Z_COG_PARAMS_ALPHA2,Z_COG_PARAMS_CHI2,ELLIPSEBIT,DM_Stellar,e_DM_lo_Stellar,e_DM_hi_Stellar,Stellar_Catalog,DM1_SN,e_DM1_SN,DM2_SN,e_DM2_SN,DM3_SN,e_DM3_SN,DM4_SN,e_DM4_SN,DM5_SN,e_DM5_SN,DM6_SN,e_DM6_SN,DM7_SN,e_DM7_SN,SN_Catalog
int64,bytes16,bytes29,int64,float64,float64,bytes21,float32,float32,float32,float32,float32,float32,bool,bytes13,int64,bytes35,int16,bool,float64,float64,float32,bytes8,float64,float64,float32,bytes4,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,float64,float64,float64,str16,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,int64,str24
2000090,SGA-2020 2000090,KKH86,2807150,208.639395,4.244438,dIrr,177.0,1.12,0.61,0.00096053514,25.974571,17.1,False,LGDWARFS,528594,KKH86,1,True,208.639395,4.244438,1.12,2086p042,208.63972178060317,4.244498901726164,1.0676076,SB26,170.32007,0.5827334,208.63968031066798,4.24463009280064,22.80629,22.110266,23.175814,22.7685,-1.0,-1.0,-1.0,2.1870558,7.5076385,14.265005,21.140234,26.860487,32.02823,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,21.529501,21.079931,20.850012,19.002533,18.631569,18.403715,17.85994,17.516594,17.313553,17.326685,16.972342,16.754358,17.085537,16.721113,16.503862,16.940128,16.571085,16.35761,-1.0,-1.0,-1.0,0.05646121,0.083752856,0.14392973,0.12178976,0.14702602,0.2187881,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.032627493,0.03765607,0.06573504,0.010871151,0.012353091,0.018345706,0.00635901,0.0073078596,0.011273069,0.005092679,0.0059680473,0.008832604,0.004640285,0.005482721,0.008107432,0.0044362904,0.0052330135,0.007900765,16.524176,1.0284809,5.574703,2.0700536,0.8340654,16.116747,0.9855422,6.54034,2.072152,0.18118383,15.921342,0.9220686,7.505497,2.1670828,1.8919169,0,27.06,0.16,0.16,McConnachi,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0

In [88]:
# Change use_cached = False if we want to redo the query at some point.
use_cached = True

if os.path.exists('cache/tf_targets_everest.fits') and use_cached:
    tf_targets = Table.read('cache/tf_targets_everest.fits')
else:
    try:
        db = psycopg2.connect(host='decatdb.lbl.gov', database='desi', user='desi')
        cursor = db.cursor()
        
        query = """SELECT DISTINCT pr.pvtype, pr.sga_id, fd.targetid, fd.fiber_ra, fd.fiber_dec, fd.fiberstatus, fd.tileid, zd.target_ra, zd.target_dec, zd.z, zd.zerr, zd.spectype, zd.deltachi2, zd.zwarn
                   FROM everest.ztile_cumulative_fibermap fd
                   INNER JOIN
                       everest.ztile_cumulative_redshifts zd ON zd.targetid = fd.targetid
                   INNER JOIN pv.proposals pr ON (CAST(pr.ra*10000 AS INTEGER) = CAST(zd.target_ra*10000 AS INTEGER)) AND 
                                                 (CAST(pr.dec*10000 AS INTEGER) = CAST(zd.target_dec*10000 AS INTEGER))
                   WHERE (pr.pvtype LIKE 'TFT' or pr.pvtype LIKE 'EXT' or pr.pvtype LIKE 'SGA');"""
        
        cursor.execute(query)
        rows = cursor.fetchall()
        tf_targets = Table(list(map(list, zip(*rows))), 
                           names=['PVTYPE', 'SGA_ID', 'TARGETID', 'FIBER_RA', 'FIBER_DEC', 'FIBERSTATUS', 'TILEID', 
                                  'TARGET_RA', 'TARGET_DEC', 'Z', 'ZERR', 'SPECTYPE', 'DELTACHI2', 'ZWARN'])
        tf_targets.write('cache/tf_targets_everest.fits', overwrite=True)
        
    except (Exception, psycopg2.errors) as error:
        print(error)
    finally:
        if db is not None:
            db.close()
            
tf_targets

PVTYPE,SGA_ID,TARGETID,FIBER_RA,FIBER_DEC,FIBERSTATUS,TILEID,TARGET_RA,TARGET_DEC,Z,ZERR,SPECTYPE,DELTACHI2,ZWARN
bytes3,int64,int64,float64,float64,int64,int64,float64,float64,float64,float64,bytes6,float64,int64
EXT,17643,39633062056692288,0.0,0.0,0,80618,115.163713024568,39.2330431988692,0.0112272729370015,2.54691135614028e-06,GALAXY,199710.72255059,0
EXT,17643,39633062056692288,115.163671928495,39.2330481232594,0,80618,115.163713024568,39.2330431988692,0.0112272729370015,2.54691135614028e-06,GALAXY,199710.72255059,0
EXT,17643,39633062056692288,115.163676763292,39.2330716810911,0,80618,115.163713024568,39.2330431988692,0.0112272729370015,2.54691135614028e-06,GALAXY,199710.72255059,0
EXT,17643,39633062056692288,115.163687172584,39.2330595735952,0,80618,115.163713024568,39.2330431988692,0.0112272729370015,2.54691135614028e-06,GALAXY,199710.72255059,0
EXT,17643,39633062056692288,115.163687272099,39.2330517354585,0,80618,115.163713024568,39.2330431988692,0.0112272729370015,2.54691135614028e-06,GALAXY,199710.72255059,0
EXT,17643,39633062056692288,115.163687614895,39.2330239448729,0,80618,115.163713024568,39.2330431988692,0.0112272729370015,2.54691135614028e-06,GALAXY,199710.72255059,0
EXT,17643,39633062056692288,115.163692746003,39.2330238257014,0,80618,115.163713024568,39.2330431988692,0.0112272729370015,2.54691135614028e-06,GALAXY,199710.72255059,0
EXT,17643,39633062056692288,115.163702566258,39.2330592761066,0,80618,115.163713024568,39.2330431988692,0.0112272729370015,2.54691135614028e-06,GALAXY,199710.72255059,0
EXT,17643,39633062056692288,115.163702811737,39.2330394673576,0,80618,115.163713024568,39.2330431988692,0.0112272729370015,2.54691135614028e-06,GALAXY,199710.72255059,0


In [89]:
l = np.unique(SGA_match)
match = []
SGA_ids = list(tf_targets['SGA_ID'])
for l_id in l:
    for SGA_id in SGA_ids:
        if SGA_id == l_id:
            # print(SGA_id)
            match.append(SGA_id)

In [91]:
len(np.unique(match))

12

## Testing Multiple values of Distance Moduli

In [23]:
S_dist = sntab['S_mu0']
S_err = sntab['S_emu0']
S2_dist = sntab['S2_mu0']
S2_err = sntab['S2_emu0']
M31_dist = sntab['M31_mu0']
M31_err = sntab['M31_emu0']
M17_dist = sntab['M17_mu0']
M17_err = sntab['M17_emu0']

NameError: name 'sntab' is not defined

In [ ]:
plt.figure(figsize = (13, 4))
plt.errorbar(range(len(sntab)), S_dist, fmt = '.', yerr = S_err, label = 'SALT', alpha = 0.2)
plt.errorbar(range(len(sntab)), S2_dist, fmt = '.', yerr = S2_err, label = 'SALT2', alpha = 0.2)
plt.errorbar(range(len(sntab)), M31_dist, fmt = '.', yerr = M31_err, label = 'M31', alpha = 0.2)
plt.errorbar(range(len(sntab)), M17_dist, fmt = '.', yerr = M17_err, label = 'M17', alpha = 0.2)
plt.legend()

In [ ]:
plt.errorbar(S_dist, S2_dist, fmt = '.', xerr = S_err, yerr = S2_err, alpha = 0.2)
plt.xlabel('SALT')
plt.ylabel('SALT2')
plt.plot([34.5, 38.5], [34.5, 38.5], 'k:');

In [ ]:
plt.errorbar(S_dist, M31_dist, fmt = '.', xerr = S_err, yerr = M31_err, alpha = 0.2)
plt.xlabel('SALT')
plt.ylabel('M31')
plt.plot([34.5, 38.5], [34.5, 38.5], 'k:');

In [ ]:
plt.errorbar(S2_dist, M31_dist, fmt = '.', xerr = S2_err, yerr = M31_err, alpha = 0.2)
plt.xlabel('SALT2')
plt.ylabel('M31')
plt.plot([33, 38.5], [33, 38.5], 'k:');

In [ ]:
plt.errorbar(M17_dist, M31_dist, fmt = '.', xerr = M17_err, yerr = M31_err, alpha = 0.2)
plt.xlabel('M17')
plt.ylabel('M31')
plt.plot([33, 38.5], [33, 38.5], 'k:');